# Learning Liar's dice with deep Q-learning

<ins>Author:</ins> Alexandre Forestier--Foray  (https://github.com/Aff54)  
<ins>Date:</ins> 22/01/2026

<ins>Note:</ins>
- For simplicity, examples are shown with 3 players and 2 dice per player by default. It is possible to increase these numbers but be aware that more players/dice can increase computation time significantly.

In [1]:
# ---- Regular Packages ----
import numpy as np
from tqdm import tqdm
import matplotlib.pyplot as plt
from matplotlib.patches import Patch
import os
from collections import namedtuple
import pandas as pd

# Reinforcement learning.
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torch.optim.lr_scheduler as lr_scheduler

In [2]:
# ---- Custom packages ----

from src.reinforcement_learning import ReplayMemory
from src.action_management import get_legal_actions_mask, get_possible_actions
from src.deterministic_agents import agent_random, agent_max_probability, agent_min_probability, Agent
from src.rl_agents import RLAgent
from src.game import GameRL
from src.performance_analysis.analysis_tools import test_agents, get_player_score
from src.performance_analysis.visual_tools import plot_pie_charts
from src.rl_agents import RLAgentOnline, EpsilonScheduler

In [ ]:
# GPU usage.
device = torch.device(
    "cuda" if torch.cuda.is_available() else
    "mps" if torch.backends.mps.is_available() else
    "cpu"
)
print(f"used divice: {device}")

# 1. Liar's dice game class

<ins>Note:</ins>
- The Game class implements the core mechanics of Liar’s Dice. It is responsible for managing player hands, turn order and interactions (calling "liar" or "exact"). It provides an interface for simulating a game.
- GameRL subclass extends Game by adding reinforcement learning oriented methods.

In [8]:
# ---- Initialisation ----

game = GameRL(player_number = 3, max_dice = 2)

print(f"""" 
----------------------------------------
hands: {[hand.tolist() for hand in game.player_hands]}
----------------------------------------
player order: {game.active_players} (left to right)
----------------------------------------
""")

" 
----------------------------------------
hands: [[4, 6], [6, 3], [6, 5]]
----------------------------------------
player order: deque([2, 3, 1]) (left to right)
----------------------------------------



In [11]:
# getting turn info
n_dice, last_bet, turn_player, turn_player_hand = game.get_turn_info()

print(f"""" 
----------------------------------------
turn player: {turn_player}
----------------------------------------
turn player hand: {turn_player_hand}
----------------------------------------
last bet: {last_bet}
----------------------------------------
""")

" 
----------------------------------------
turn player: 2
----------------------------------------
turn player hand: [6, 3]
----------------------------------------
last bet: [1, 0]
----------------------------------------



In [12]:
# ---- Playing from a player's perspective ----

# insert your action here. Format: [quantity, value]
# calling "liar":[-1, -1] / "exact": [0, 0]
action = [2, 2]

game.make_a_bet(action, verbose = True)

# getting turn info
n_dice, last_bet, turn_player, turn_player_hand = game.get_turn_info()

print(f"""" 
----------------------------------------
turn player: {turn_player}
----------------------------------------
turn player hand: {turn_player_hand}
----------------------------------------
last bet: {last_bet}
----------------------------------------
""")

" 
----------------------------------------
turn player: 3
----------------------------------------
turn player hand: [6, 5]
----------------------------------------
last bet: [2, 2]
----------------------------------------

